In [35]:
%pylab inline
import tensorflow as tf

Populating the interactive namespace from numpy and matplotlib


In [36]:
from fuel.datasets.cifar10 import CIFAR10
from fuel.transformers import ScaleAndShift, Cast, Flatten, Mapping
from fuel.streams import DataStream
from fuel.schemes import SequentialScheme, ShuffledScheme


CIFAR10.default_transformers = (
    (ScaleAndShift, [2.0 / 255.0, -1], {'which_sources': 'features'}),
    (Cast, [np.float32], {'which_sources': 'features'}), 
    (Flatten, [], {'which_sources': 'features'}),
    (Mapping, [lambda batch: (b for b in batch)], {})
)

In [3]:
cifar10_train = CIFAR10(("train",), subset=slice(None, 40000))

#this stream will shuffle the CIFAR10 set and return us batches of 100 examples
cifar10_train_stream = DataStream.default_stream(
    cifar10_train,
    iteration_scheme=ShuffledScheme(cifar10_train.num_examples, 25)
)

cifar10_validation = CIFAR10(("train",), subset=slice(40000, None))

In [4]:
# cifar10_train = CIFAR10(("train",))

# #this stream will shuffle the CIFAR10 set and return us batches of 100 examples
# cifar10_train_stream = DataStream.default_stream(
#     cifar10_train,
#     iteration_scheme=ShuffledScheme(cifar10_train.num_examples, 100)
# )

In [5]:
# We will use larger portions for testing and validation
# as these dont do a backward pass and reauire less RAM.
cifar10_validation_stream = DataStream.default_stream(
    cifar10_validation, 
    iteration_scheme=SequentialScheme(cifar10_validation.num_examples, 100)
)

cifar10_test = CIFAR10(("test",))
cifar10_test_stream = DataStream.default_stream(
    cifar10_test,
    iteration_scheme=SequentialScheme(cifar10_test.num_examples, 100)
)

In [6]:
print "The streams return batches containing %s" % (cifar10_train_stream.sources,)

print "Each trainin batch consits of a tuple containing:"
for element in next(cifar10_train_stream.get_epoch_iterator()):
    print " - an array of size %s containing %s" % (element.shape, element.dtype)
    
print "Validation/test batches consits of tuples containing:"
for element in next(cifar10_test_stream.get_epoch_iterator()):
    print " - an array of size %s containing %s" % (element.shape, element.dtype)

The streams return batches containing (u'features', u'targets')
Each trainin batch consits of a tuple containing:
 - an array of size (25, 3072) containing float32
 - an array of size (25, 1) containing uint8
Validation/test batches consits of tuples containing:
 - an array of size (100, 3072) containing float32
 - an array of size (100, 1) containing uint8


In [7]:
x = tf.placeholder(tf.float32, shape=[None, 3072])
y_ = tf.placeholder(tf.int64, shape=[None, 10])
W = tf.Variable(tf.zeros([3072, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.matmul(x, W) + b

In [8]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [9]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [10]:
conv1_f = 32
conv2_f = 64
# conv3_f = 128
fc1_w = 1024
fc2_w = 10

af_conv = 8

In [11]:
x_image = tf.reshape(x, [-1, 32, 32, 3])

In [12]:
W_conv1 = weight_variable([5, 5, 3, conv1_f])
b_conv1 = bias_variable([conv1_f])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
print h_pool1.shape

(?, 16, 16, 32)


In [13]:
W_conv2 = weight_variable([5, 5, conv1_f, conv2_f])
b_conv2 = bias_variable([conv2_f])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
print h_pool2.shape

(?, 8, 8, 64)


In [14]:
# W_conv3 = weight_variable([5, 5, conv2_f, conv3_f])
# b_conv3 = bias_variable([conv3_f])

# h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
# h_pool3 = max_pool_2x2(h_conv3)

In [15]:
W_fc1 = weight_variable([af_conv * af_conv * conv2_f, fc1_w])
b_fc1 = bias_variable([fc1_w])

h_pool2_flat = tf.reshape(h_pool2, [-1, af_conv * af_conv * conv2_f])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [16]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [17]:
W_fc2 = weight_variable([fc1_w, fc2_w])
b_fc2 = bias_variable([fc2_w])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [18]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv)
)
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [19]:
# counter = 0

# try:
#     for b in cifar10_train_stream.get_epoch_iterator():
#         print "batch 1"
#         while 1:
#             print 'n', next(b)
#         counter += 1
#     print counter
# except:
#     print counter, sys.exc_info()[0]

In [20]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
validation_errors = []

for i in range(50000):
    batch = next(cifar10_train_stream.get_epoch_iterator())
    X = next(batch)
    Y = next(batch)
    # Y = np.array(np.arange(10, 100) == Y[0], dtype=np.uint8)
    Y = np.array(np.arange(10) == Y, dtype=np.uint8)

    if i % 100 == 0:
        train_accuracy = accuracy.eval(
            feed_dict={x: X, y_: Y, keep_prob: 1.0}
        )
        print("step %d, training accuracy %g" % (i, train_accuracy))
        
    if i % 1000 == 0:
        val_acc = []
        for v_X, v_Y in cifar10_validation_stream.get_epoch_iterator():
            v_Y = np.array(np.arange(10) == v_Y, dtype=np.uint8)
            acc = accuracy.eval(feed_dict={x: v_X, y_: v_Y, keep_prob: 1.0})
            val_acc.append(acc)
        print("     %d, valid accuracy: %g" % (i, np.mean(val_acc)))
        
    train_step.run(feed_dict={x: X, y_: Y, keep_prob: 0.5})
    
    

# print("test accuracy %g" % accuracy.eval(
#     feed_dict={x: cifar10.test.images, y_: cifar10.test.labels}
# ))

step 0, training accuracy 0.92
     0, valid accuracy: 0.6214
step 100, training accuracy 1
step 200, training accuracy 0.96
step 300, training accuracy 0.92
step 400, training accuracy 1
step 500, training accuracy 0.92
step 600, training accuracy 1
step 700, training accuracy 0.92
step 800, training accuracy 0.92
step 900, training accuracy 0.96
step 1000, training accuracy 0.92
     1000, valid accuracy: 0.6178
step 1100, training accuracy 0.96
step 1200, training accuracy 0.88
step 1300, training accuracy 0.92
step 1400, training accuracy 0.96
step 1500, training accuracy 0.88
step 1600, training accuracy 0.96
step 1700, training accuracy 0.92
step 1800, training accuracy 1
step 1900, training accuracy 0.92
step 2000, training accuracy 0.96
     2000, valid accuracy: 0.6133
step 2100, training accuracy 0.92
step 2200, training accuracy 0.84
step 2300, training accuracy 0.8
step 2400, training accuracy 1
step 2500, training accuracy 0.76
step 2600, training accuracy 0.84
step 2700, 

In [40]:
test_accuracy = []
for X, Y in cifar10_test_stream.get_epoch_iterator():
    Y = np.array(np.arange(10) == Y, dtype=np.uint8)
    test_acc = accuracy.eval(feed_dict={x: X, y_: Y, keep_prob: 1.0})
    # print("step %d, test accuracy %g" % (len(test_accuracy), test_acc))
    test_accuracy.append(test_acc)
    
print("test accuracy %g" % np.mean(test_accuracy))

test accuracy 0.6169


In [ ]:
# subplot(2,1,1)
# train_loss = np.array(train_loss)
# semilogy(train_loss[:,0], train_loss[:,1], label='batch train loss')
# legend()

# subplot(2,1,2)
# train_erros = np.array(train_erros)
# plot(train_erros[:,0], train_erros[:,1], label='batch train error rate')
# validation_errors = np.array(validation_errors)
# plot(validation_errors[:,0], validation_errors[:,1], label='validation error rate', color='r')
# ylim(0,0.2)
# legend()